In [9]:
# 显示cell运行时长
%load_ext klab-autotime

The klab-autotime extension is already loaded. To reload it, use:
  %reload_ext klab-autotime
time: 1.26 ms


In [10]:
%env USER=kesci

env: USER=kesci
time: 1.05 ms


In [11]:
import numpy as np
import pandas as pd
import xgboost as xgb
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import catboost as ctb

time: 1.76 ms


In [4]:
def get_rate(data):
    data['type0_rate'] = data['act_type0_count']/(data['act_day_count']+0.00001)
    data['type1_rate'] = data['act_type1_count']/(data['act_day_count']+0.00001)
    data['type2_rate'] = data['act_type2_count']/(data['act_day_count']+0.00001)
    data['type3_rate'] = data['act_type3_count']/(data['act_day_count']+0.00001)
    
    data['page0_rate'] = data['act_page0_count']/(data['act_day_count']+0.00001)
    data['page1_rate'] = data['act_page1_count']/(data['act_day_count']+0.00001)
    data['page2_rate'] = data['act_page2_count']/(data['act_day_count']+0.00001)
    data['page3_rate'] = data['act_page3_count']/(data['act_day_count']+0.00001)
    data['page4_rate'] = data['act_page4_count']/(data['act_day_count']+0.00001)
    
    act_mean = data['act_day_count'].mean()
    data['is_have_act'] = data['act_day_count'].apply(lambda x:0 if x<=0 else (1 if x < act_mean else 2))
    video_mean = data['video_day_count'].mean()
    data['is_have_video'] = data['video_day_count'].apply(lambda x:0 if x<=0 else (1 if x < video_mean else 2))
    type0_mean = data['act_type0_count'].mean()
    data['is_have_type0'] = data['act_type0_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type0_mean else 2))
    type1_mean = data['act_type1_count'].mean()
    data['is_have_type1'] = data['act_type1_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type1_mean else 2))
    type2_mean = data['act_type2_count'].mean()
    data['is_have_type2'] = data['act_type2_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type2_mean else 2))
    
    act_mean = data['act_day_count'].mean()
    data['act_day_count1'] = data['act_day_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < act_mean else 2))
    
    act_max = data['act_day_max'].mean()
    data['act_day_max1'] = data['act_day_max'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < act_max else 2))
    data['act_day_max2'] = data['act_day_max'].fillna(0).apply(lambda x:0 if x<=0 else 1)
    # page2_mean = data['act_page2_count'].mean()
    # data['is_have_page2'] = data['act_page2_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < page2_mean else 2))
    data['every_video_rate'] = data['act_day_count']/(data['act_video_nums']+0.00001)
    data['every_video_author_rate'] = data['act_author_counts']/(data['act_video_nums']+0.00001)
    # data['author_act_rate'] =data['act_author_counts']/data['act_day_count']
    
    data['act_daily_max_rate'] = data['act_daily_max']/(data['act_day_count']+0.00001)
    data['act_single_video_rate'] = data['single_video_max']/(data['act_day_count']+0.00001)
    # data['app_daily_max_rate'] = data['app_daily_max']/(data['app_day_count']+0.00001)
    
    # data['every_page_type_rate'] = data['every_page_type_max']/data['act_daily_max']
    # data['every_video_type_rate'] = data['every_video_type_max']/data['act_daily_max']
    
    data['app_act_rate'] = data['app_day_std']/(data['act_last1days_counts']+0.00001)
    data['app_video_type_rate'] = data['app_day_max']/(data['every_video_type_mean']+0.00001)
    data['app_video_type_sub'] = data['app_day_max']-data['every_video_type_mean']
    
    data['page1_2_rate'] = data['page2_rate']/(data['page1_rate']+0.00001)
    data['app_std_mul'] = data['app_day_std']*data['page1_rate']
    data['act_last1_mul'] = data['act_last1days_counts']*data['page1_rate']
    data['act_last1_std_mul'] = (data['act_last1days_counts']+data['app_day_std'])*data['page1_rate']
    
    data['app_diff_rate'] = data['app_day_diff_std']/(data['app_day_std']+0.00001)
    data['app_page1_rate'] = data['app_day_diff_std']*data['page1_rate']
    data['page1_type2_rate'] = data['page1_rate']*data['type2_rate']
    
    data['app_std_div_mean'] = data['app_day_std']/(data['app_day_mean']+0.00001)
    data['act_std_div_mean'] = data['act_day_std']/(data['act_day_mean']+0.00001)
    data['video_std_div_mean'] = data['video_day_std']/(data['video_day_mean']+0.00001)
    
    data['register_day'] = data['register_day'] + 1
    # data['register_day'] = data['register_day'].apply(lambda x:16 if x>16 else x)
    
    data['device_type1'] = data['device_type'].apply(lambda x:0 if x< 85 else (1 if x<170 else 2))
    
    data['act_continue_std_div_mean'] = data['act_continue_std']/(data['act_continue_mean']+0.00001)
    data['app_continue_std_div_mean'] = data['app_continue_std']/(data['app_continue_mean']+0.00001)
    
    data['app_div_reg'] = data['app_day_count']/data['register_day']
    data['act_div_reg'] = data['act_day_count']/data['register_day']
    data['video_div_reg'] = data['video_day_count']/data['register_day']
    data['author_div_reg'] = data['act_author_count']/data['register_day']
    data['act_video_div_reg'] = data['act_video_nums']/data['register_day']
    
    data['act_last1days_div_reg'] = data['act_last1days_counts']/data['register_day']
    
    # data['act_author_q3_q1'] = data['act_author_q3']-(data['act_author_q1']+0.00001)
    # data['act_author_q2'] = data['act_author_q1']*2
    # data['act_author_q4'] = data['act_author_q3']+data['act_author_q1']
    
    # data['every_video_type_std_mean'] = data['every_video_type_std']+data['every_video_type_mean']
    data['device_exp_reg'] = data['device_type']/(data['register_type']+0.00001)
    
    data['video_max_min'] = data['video_day_max']-data['video_day_min']
    data['app_max_min'] = data['app_day_max']-data['app_day_min']
    data['act_max_min'] = data['act_day_max']-data['act_day_min']
    
    data['act_app_day_exp'] = data['app_day_max']/(data['act_day_max']+0.00001)
    data['video_app_day_exp'] = data['app_day_max']/(data['video_day_max']+0.00001)
    
    data['app_day_diff_std_mean'] = data['app_day_diff_std']/(data['app_day_diff_mean']+0.00001)
    data['app_day_std_mean'] = data['app_day_std']/(data['app_day_mean']+0.00001)
    
    data['act_app_day_mul'] = data['app_day_max']+data['act_day_max']
    data['video_app_day_mul'] = data['app_day_max']+data['video_day_max']
    data['video_act_day_mul'] = data['act_day_max']+data['video_day_max']
    
    data['author_rate'] = data['act_author_count']/data['act_author_counts']
    
    data['act_author_rate'] = data['act_author_counts']/data['act_day_count']
    data['app_author_rate'] = data['act_author_counts']/data['app_day_count']
    data['video_author_rate'] = data['act_author_counts']/data['video_day_count']
    
    return data

time: 19.7 ms


In [5]:
def xgb_train(train,test):
    # print(train.columns.values.tolist())
    # print(len(train.columns.values.tolist()))
    
    test_feature = test.drop("user_id",1)
    
    l = pd.DataFrame()
    l['label'] = train['label']
    l = l.groupby('label',as_index=False)['label'].agg({'label_count':'count'})
    print(l)
    
    label = train['label']      
    train.drop(['user_id','label'], inplace=True, axis=1)
    
    used_feature =[
        
        'act_author_count', 'act_author_counts', 'act_continue_mean', 
        'act_continue_std', 'act_day_mean', 'act_day_min',  
        'act_last1days_counts', 'act_last2days_counts', 'act_last3days_counts', 'act_last4days_counts', 
        'act_last5days_counts', 'act_last6days_counts', 'act_last7days_counts', 'act_last_gap', 'act_page0_count', 
        'act_page1_count', 'act_page2_count', 'act_page3_count', 'act_page4_count', 'act_type0_count', 
        'act_type1_count', 'act_type2_count', 'act_type3_count', 
        'app_author_rate',  'app_continue_std', 'app_day_count', 
        'app_day_diff_max', 'app_day_diff_mean', 'app_day_diff_std',
        'app_day_max', 'app_day_mean', 'app_day_std',
        'app_last3days_counts', 'app_last4days_counts', 'app_last5days_counts', 'app_last6days_counts', 
        'app_last7days_counts', 'app_last_gap', 'author_rate', 'device_type', 'page0_rate', 'page1_rate',
        'page2_rate', 'register_day', 'register_type', 'type0_rate', 
        'type1_rate', 'type2_rate', 'video_author_rate', 
        'video_day_mean', 'video_day_std',
        'video_last7days_counts','act_single_video_rate', 'act_app_day_exp', 'video_app_day_exp','every_video_type_mean', 
        'every_page_type_mean', 'act_daily_max', 'act_video_nums', 'single_video_max',
        'app_act_rate','app_video_type_rate','app_video_type_sub',
        
        'device_exp_reg',
        'video_app_day_mul','act_app_day_mul','video_act_day_mul',
        'is_have_act','is_have_video','is_have_type2','is_have_type1','is_have_type0',
        'app_act_rate','act_daily_max_rate',
        'app_day_skew','app_day_kurt','act_day_skew','act_day_kurt',
        'act_nums','video_nums','act_day_sum','app_day_sum',
        'device_type_register_rate',
    ]
    t = train[used_feature]
    text_use_feature = test[used_feature]
    
    N = 5
    skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=6666)
    
    xx_cv = []
    xx_pre = []
    train_value = t.values
    label = label.values
    test_feature = text_use_feature.values
    
    train_features = t.columns.tolist()
    df = pd.DataFrame(train_features, columns=['feature'])
    df1 = pd.DataFrame(train_features, columns=['feature'])
    imp1 = []
    i = 0
    for train_in,test_in in skf.split(train_value,label):
        print("折数",i+1)
        X_train,X_test,y_train,y_test = train_value[train_in],train_value[test_in],label[train_in],label[test_in]
        
        xgb_val = xgb.DMatrix(X_test,label=y_test)
        xgb_train = xgb.DMatrix(X_train,label=y_train)
        xgb_test = xgb.DMatrix(test_feature)
    
        params = {
            'boosting_type': 'gbtree',
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'gamma':0.1,
            'max_depth': 5,
            'min_child_weight': 3,
            'eta': 0.05,
            'silent': 0,
            'subsample': 0.8,
            'alpha': 0.6,  
            'lambda': 0.001,  # 越小l2正则程度越高
            'colsample_bytree': 0.5,
        }
        
        plst = list(params.items())
        num_rounds = 20000  # 迭代次数
        watchlist = [(xgb_train, 'train'), (xgb_val, 'val')]
        
        print('开始训练......')
        gbm = xgb.train(
           plst,
           xgb_train,
           num_rounds,
           watchlist,
           early_stopping_rounds=300,
           verbose_eval=300
       )
        X_test=xgb.DMatrix(X_test)
        y_pred = gbm.predict(X_test)
        # y_test=pd.Series(y_test)
        xx_cv.append(roc_auc_score(y_test,y_pred))
        xx_pre.append(gbm.predict(xgb_test))
        i = i+1
    
    print('xx_cv',np.mean(xx_cv))
    s = 0
    for i in xx_pre:
        s = s + i
    s = s/N
    df_result = pd.DataFrame()
    df_result['user_id'] = test['user_id']
    df_result['probability'] = list(s)
    time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
    df_result.to_csv('/home/kesci/bigdata/result/xgb_%s_%s.csv'%(str(time_date),str(np.mean(xx_cv)).split('.')[1]),sep=',',index=False,header=None)
    return s

time: 25.7 ms


In [29]:
f1 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck1_feature.csv")
f1 = get_rate(f1)

f2 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck2_feature.csv")
f2 = get_rate(f2)

f = pd.concat([f1,f2],axis=0,ignore_index=True,join='outer')

f3 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck3_feature.csv")
f3 = get_rate(f3)
s = xgb_train(f,f3)
test_label = f2['label']
print("线下验证--》",roc_auc_score(test_label,s))

   label  label_count
0      0       339024
1      1       339220
折数 1
开始训练......
[0]	train-auc:0.877938	val-auc:0.876982
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 300 rounds.
[300]	train-auc:0.891767	val-auc:0.888887
[600]	train-auc:0.894536	val-auc:0.889195
[900]	train-auc:0.896808	val-auc:0.889216
Stopping. Best iteration:
[821]	train-auc:0.896243	val-auc:0.88923

折数 2
开始训练......
[0]	train-auc:0.878105	val-auc:0.877913
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 300 rounds.
[300]	train-auc:0.891609	val-auc:0.889423
[600]	train-auc:0.894244	val-auc:0.889698
[900]	train-auc:0.896635	val-auc:0.88973
Stopping. Best iteration:
[759]	train-auc:0.895521	val-auc:0.889753

折数 3
开始训练......
[0]	train-auc:0.878086	val-auc:0.877857
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train unt

In [31]:
def cat_train(train,test):
    # print(train.columns.values.tolist())
    # print(len(train.columns.values.tolist()))
    
    test_feature = test.drop("user_id",1)
    
    l = pd.DataFrame()
    l['label'] = train['label']
    l = l.groupby('label',as_index=False)['label'].agg({'label_count':'count'})
    print(l)
    
    label = train['label']      
    train.drop(['user_id','label'], inplace=True, axis=1)
    
    used_feature =[
        'act_author_count', 'act_author_counts', 'act_continue_mean', 
        'act_continue_std', 'act_day_mean', 'act_day_min',  
        'act_last1days_counts', 'act_last2days_counts', 'act_last3days_counts', 'act_last4days_counts', 
        'act_last5days_counts', 'act_last6days_counts', 'act_last7days_counts', 'act_last_gap', 'act_page0_count', 
        'act_page1_count', 'act_page2_count', 'act_page3_count', 'act_page4_count', 'act_type0_count', 
        'act_type1_count', 'act_type2_count', 'act_type3_count', 
        'app_author_rate',  'app_continue_std', 'app_day_count', 
        'app_day_diff_max', 'app_day_diff_mean', 'app_day_diff_std',
        'app_day_max', 'app_day_mean', 'app_day_std',
        'app_last3days_counts', 'app_last4days_counts', 'app_last5days_counts', 'app_last6days_counts', 
        'app_last7days_counts', 'app_last_gap', 'author_rate', 'device_type', 'page0_rate', 'page1_rate',
        'page2_rate', 'register_day', 'register_type', 'type0_rate', 
        'type1_rate', 'type2_rate', 'video_author_rate', 
        'video_day_mean', 'video_day_std',
        'video_last7days_counts','act_single_video_rate', 'act_app_day_exp', 'video_app_day_exp','every_video_type_mean', 
        'every_page_type_mean', 'act_daily_max', 'act_video_nums', 'single_video_max',
        'app_act_rate','app_video_type_rate','app_video_type_sub',
        
        'device_exp_reg',
        'video_app_day_mul','act_app_day_mul','video_act_day_mul',
        'is_have_act','is_have_video','is_have_type2','is_have_type1','is_have_type0',
        'app_act_rate','act_daily_max_rate',
        'app_day_skew','app_day_kurt','act_day_skew','act_day_kurt',
        'act_nums','video_nums','act_day_sum','app_day_sum',
        'device_type_register_rate',
        ]
    t = train[used_feature]
    text_use_feature = test[used_feature]
    
    N = 5
    skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=6666)
    
    xx_cv = []
    xx_pre = []
    train_value = t.values
    label = label.values
    test_feature = text_use_feature.values
    
    i = 0
    for train_in,test_in in skf.split(train_value,label):
        print("折数",i+1)
        X_train,X_test,y_train,y_test = train_value[train_in],train_value[test_in],label[train_in],label[test_in]
        
        dtrain = ctb.Pool(X_train,y_train)
        dvalid = ctb.Pool(X_test,y_test)  

        model = ctb.CatBoostClassifier(
            thread_count = 8, 
            iterations = 10000,
            depth=4, 
            learning_rate=0.05,
            eval_metric='AUC',
            verbose=300,
            od_type = 'Iter',
            # l2_leaf_reg=0.001,
            od_wait = 150)
            
        model.fit(dtrain, eval_set=dvalid)
        y_pred = model.predict_proba(X_test)[:,1]
        # print("pred-->",model.predict_proba(X_test))
        xx_cv.append(roc_auc_score(y_test,y_pred))
        xx_pre.append(model.predict_proba(test_feature)[:,1])
        i = i+1
        
    print('xx_cv',np.mean(xx_cv))
    s = 0
    for i in xx_pre:
        s = s + i
    s = s/N
    
    df_result = pd.DataFrame()
    df_result['user_id'] = test['user_id']
    df_result['probability'] = list(s)
    time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
    df_result.to_csv('/home/kesci/bigdata/result/cat_%s_%s.csv'%(str(time_date),str(np.mean(xx_cv)).split('.')[1]),sep=',',index=False,header=None)
    return s

time: 21.1 ms


In [32]:
f1 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck1_feature.csv")
f1 = get_rate(f1)

f2 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck2_feature.csv")
f2 = get_rate(f2)

f = pd.concat([f1,f2],axis=0,ignore_index=True,join='outer')

f3 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck3_feature.csv")
f3 = get_rate(f3)
s = cat_train(f,f3)
# test_label = f2['label']
# print("线下验证--》",roc_auc_score(test_label,s))

   label  label_count
0      0       339024
1      1       339220
折数 1
0:	learn: 0.8622341	test: 0.8614123	best: 0.8614123 (0)	total: 234ms	remaining: 38m 57s
300:	learn: 0.8882045	test: 0.8873415	best: 0.8873415 (300)	total: 1m 32s	remaining: 49m 44s
600:	learn: 0.8893587	test: 0.8881873	best: 0.8881873 (600)	total: 3m 9s	remaining: 49m 25s
900:	learn: 0.8900376	test: 0.8885240	best: 0.8885240 (900)	total: 4m 44s	remaining: 47m 48s
1200:	learn: 0.8905604	test: 0.8886962	best: 0.8886962 (1200)	total: 6m 18s	remaining: 46m 14s
1500:	learn: 0.8909998	test: 0.8887819	best: 0.8887853 (1482)	total: 7m 53s	remaining: 44m 41s
1800:	learn: 0.8913896	test: 0.8888439	best: 0.8888459 (1769)	total: 9m 28s	remaining: 43m 6s
2100:	learn: 0.8917736	test: 0.8888955	best: 0.8888972 (2027)	total: 11m 2s	remaining: 41m 30s
2400:	learn: 0.8921212	test: 0.8889255	best: 0.8889269 (2371)	total: 12m 36s	remaining: 39m 54s
2700:	learn: 0.8924478	test: 0.8889614	best: 0.8889626 (2691)	total: 14m 10s	remaining: 

In [12]:
def xgb_lr_train(train,test):
    # print(train.columns.values.tolist())
    # print(len(train.columns.values.tolist()))
    
    col =[
        'act_author_count', 'act_author_counts', 'act_continue_mean', 
        'act_continue_std', 'act_day_mean', 'act_day_min',  
        'act_last1days_counts', 'act_last2days_counts', 'act_last3days_counts', 'act_last4days_counts', 
        'act_last5days_counts', 'act_last6days_counts', 'act_last7days_counts', 'act_last_gap', 'act_page0_count', 
        'act_page1_count', 'act_page2_count', 'act_page3_count', 'act_page4_count', 'act_type0_count', 
        'act_type1_count', 'act_type2_count', 'act_type3_count', 
        'app_author_rate',  'app_continue_std', 'app_day_count', 
        'app_day_diff_max', 'app_day_diff_mean', 'app_day_diff_std',
        'app_day_max', 'app_day_mean', 'app_day_std',
        'app_last3days_counts', 'app_last4days_counts', 'app_last5days_counts', 'app_last6days_counts', 
        'app_last7days_counts', 'app_last_gap', 'author_rate', 'device_type', 'page0_rate', 'page1_rate',
        'page2_rate', 'register_day', 'register_type', 'type0_rate', 
        'type1_rate', 'type2_rate', 'video_author_rate', 
        'video_day_mean', 'video_day_std',
        'video_last7days_counts','act_single_video_rate', 'act_app_day_exp', 'video_app_day_exp','every_video_type_mean', 
        'every_page_type_mean', 'act_daily_max', 'act_video_nums', 'single_video_max',
        'app_act_rate','app_video_type_rate','app_video_type_sub',
        
        'device_exp_reg',
        'video_app_day_mul','act_app_day_mul','video_act_day_mul',
        'is_have_act','is_have_video','is_have_type2','is_have_type1','is_have_type0',
        'act_daily_max_rate',
        'app_day_skew','app_day_kurt','act_day_skew','act_day_kurt',
        'act_nums','video_nums','act_day_sum','app_day_sum',
        'device_type_register_rate',
        ]
    X_train = train[col]
    y_train = train['label'].values
    X_test = test[col]
    # y_test = test['label'].values
    

    model = xgb.XGBClassifier(
        nthread=8,# cpu 线程数 默认最大
        n_estimators=100,
        max_depth=4,
        seed=6666,
        learning_rate=0.05,
        subsample=0.8,
        min_child_weight=3,
        colsample_bytree=.9,
        gamma = 0.05,
        reg_alpha=0.7,
        reg_lambda=0.05,
        silent=False,
        eval_metric='auc'
    )

    model.fit(X_train, y_train)
    y_pred_test = model.predict_proba(X_test)[:,1]
    # xgb_test_auc = roc_auc_score(y_test, y_pred_test)
    # print('xgboost test auc: %.5f' % xgb_test_auc)
            
    # xgboost编码原有特征
    X_train_leaves = model.apply(X_train)
    X_test_leaves = model.apply(X_test)
    
    # 合并编码后的训练数据和测试数据
    All_leaves = np.concatenate((X_train_leaves, X_test_leaves), axis=0)
    All_leaves = All_leaves.astype(np.int32)
    
    # 对所有特征进行ont-hot编码
    xgbenc = OneHotEncoder()
    X_trans = xgbenc.fit_transform(All_leaves)
    
    (train_rows, cols) = X_train_leaves.shape
    
    # 定义LR模型
    lr = LogisticRegression(n_jobs=8,max_iter=3000,C = 0.01,
          verbose=1)
    # lr对xgboost特征编码后的样本模型训练
    lr.fit(X_trans[:train_rows, :], y_train)
    # 预测及AUC评测
    y_pred_xgblr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
    # xgb_lr_auc1 = roc_auc_score(y_test, y_pred_xgblr1)
    # print('基于Xgb特征编码后的LR AUC: %.5f' % xgb_lr_auc1)
    df_result = pd.DataFrame()
    df_result['user_id'] = test['user_id']
    df_result['probability'] = y_pred_xgblr1
    time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
    df_result.to_csv('/home/kesci/bigdata/result/xgb_lr_%s.csv' %(str(time_date)), sep=',',index=False,header=None)
    

time: 21.2 ms


In [13]:
f1 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck1_feature.csv")
f1 = get_rate(f1)

f2 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck2_feature.csv")
f2 = get_rate(f2)

f = pd.concat([f1,f2],axis=0,ignore_index=True,join='outer')

f3 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck3_feature.csv")
f3 = get_rate(f3)
xgb_lr_train(f,f3)

/opt/conda/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(self.n_jobs))


[LibLinear]time: 2min 22s
